# Lung Cancer Classification using Computerized Tomography
 Project developed by: **Eduardo Passos** [202205630](https://sigarra.up.pt/fcup/pt/fest_geral.cursos_list?pv_num_unico=202205630), **Pedro Fernandes** [202208347](https://sigarra.up.pt/fcup/pt/fest_geral.cursos_list?pv_num_unico=202208347) and **Rafael Pacheco** [202206258](https://sigarra.up.pt/fcup/pt/fest_geral.cursos_list?pv_num_unico=202206258)


### Index {#index} #############################################
1. [Project Introduction](#intro)
2. [Data Understanding](#understand)
3. [Pre-Processing and Conversion](#preproc)
4. [Bagging for Feature Selection](#bagging)
5. [Exploratory Data Analysis](#eda)
6. [Models](#models)
6.1. [Support Vector Machines](#svm)


## 1. Project Introduction and Motivation {#intro}
[Back to Index](#index)

This project focuses on classifying lung cancer using Computed Tomography (CT) scan data, using the [**LIDC-IDRI**](https://wiki.cancerimagingarchive.net/pages/viewpage.action?pageId=1966254) dataset. 

As referenced in the assignment, **lung cancer leads global cancer-related mortality rates**, as only 16% of cases are diagnosed as local-stage tumours. In these instances, patients have a five-year survival rate exceeding 50%. However, when diagnosed at an advanced stage, the five-year survival rate drops to just 5%. Therefore, **early diagnosis is crucial for improving survival outcomes**, and systems designed to assist with screening can be instrumental regarding this matter (statistics from [World Health Organization](https://www.iarc.who.int/featured-news/latest-global-cancer-data-cancer-burden-rises-to-18-1-million-new-cases-and-9-6-million-cancer-deaths-in-2018/) and [American Cancer Society](https://www.cancer.org/research/cancer-facts-statistics/all-cancer-facts-figures/cancer-facts-figures-2019.html)).

The goal is to **develop a machine learning model capable of accurately identifying lung cancer** (based on the provided medical imaging data) in early stages of the tumour, in order to prevent evolution of the disease. To support this, we have developed a program that **converts the dataset** (initially in DICOM format) into numpy array files (.npy), while extracting and storing key information in a CSV file.


## 2. Data Understanding {#understand}
[Back to Index](#index)

The [LIDC-IDRI](https://wiki.cancerimagingarchive.net/pages/viewpage.action?pageId=1966254) dataset **contains the imaging data of the lungs** from over 1000 patients. This dataset is part of a collection of datasets provided by [**The Cancer Imaging Archive**](https://www.cancerimagingarchive.net/browse-collections/) for public research. The medical imaging data is provided in [**DICOM**](https://en.wikipedia.org/wiki/DICOM) (.dcm), and contains both image data, and associated metadata. It also contains the annotations of the medical professionals regarding the subject. 

The nodule annotations in the dataset **reflect the radiologist's evaluation** and include key information such as nodule size and malignancy rating (on a scale from 1-5). However, the **malignancy classification may vary based on each radiologist's judgment** and reasoning. What this means is that it introduces a level of subjectivity, making the classification potentially ambiguous and not always straightforward.

The dataset contains noise, missing data, and file corruption. Some patients have incomplete annotations or no annotations at all, which complicates the training process since not all scans contain the necessary labels for nodule classification. ???????????????????????????????/In some cases, patients may not have nodules, making their scans irrelevant for certain tasks focused on nodule detection.

## 3. Pre-Processing and Conversion {#preproc}
[Back to Index](#index)

(The following script is in the file *`preprocessing.py`*)

To convert the "raw dataset" into a format suitable for modeling, the base code was inspired by a [**GitHub repository**](https://github.com/jaeho3690/LIDC-IDRI-Preprocessing). However, a lot of changes were added for further filtering and extraction.

The following proceedings took place:

 - **Data quality assessments** ensuring that the scans and slices have complete and consistent information (such as slice thickness and valid DICOM attributes);
 - The images were **normalized by clipping the Hounsfield Unit** (HU) values between -1000 and -400, which removes bone matter and air, both irrelevant to the study;
 - The **pixel values were rescaled to a 0-1 range for consistency** across scans;
 - **Lung segmentation** was then applied to **isolate the lung regions** and remove background noise;
 - **Radiomic features were extracted to a CSV file** using PyRadiomics (?????????????????????? that will be later discussed) to capture important nodule characteristics, including texture and shape;
 - Finally, **data augmentation** was employed to **increase the dataset's variability** and create a more robust model. This was achieved through random flips, shifts, rotations, brightness adjustments, and noise addition. This process was influenced by the work in the following [article](https://paperswithcode.com/paper/procan-progressive-growing-channel-attentive).
 

The code explanation and documentation resides within the file.

## 4. Bagging for Feature Selection {#bagging}
[Back to Index](#index)


## 5. Exploratory Data Analysis {#eda}
[Back to Index](#index)

